In [1]:
from datetime import datetime
from pathlib import Path

import pandas as pd
import polars as pl
import pytz
import yfinance as yf
from dateutil import parser


Question 1

In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

In [3]:
df_pandas = pd.read_html(URL)

In [4]:
df = pl.from_pandas(df_pandas[0])

In [5]:
df = df.select(
    pl.col("Symbol").alias("ticker"),
    pl.col("Security").alias("name"),
    pl.col("Date added").cast(pl.Date).alias("date_added"),
).with_columns(pl.col("date_added").dt.year().alias("year_added"))

In [6]:
df.group_by("year_added").agg(pl.col("ticker").count().alias("ticker_count")).sort(
    ["ticker_count", "year_added"], descending=True
)[1]["year_added"].item()

2017

In [7]:
current_year = datetime.now().year
df.filter(pl.col("year_added") <= current_year - 20).select(
    pl.col("ticker").count()
).item()

226

Question 2

In [8]:
world_indices = {
    "United States": {"index": "S&P 500", "ticker": "^GSPC"},
    "China": {"index": "Shanghai Composite", "ticker": "000001.SS"},
    "Hong Kong": {"index": "HANG SENG INDEX", "ticker": "^HSI"},
    "Australia": {"index": "S&P/ASX 200", "ticker": "^AXJO"},
    "India": {"index": "Nifty 50", "ticker": "^NSEI"},
    "Canada": {"index": "S&P/TSX Composite", "ticker": "^GSPTSE"},
    "Germany": {"index": "DAX", "ticker": "^GDAXI"},
    "United Kingdom": {"index": "FTSE 100", "ticker": "^FTSE"},
    "Japan": {"index": "Nikkei 225", "ticker": "^N225"},
    "Mexico": {"index": "IPC Mexico", "ticker": "^MXX"},
    "Brazil": {"index": "Ibovespa", "ticker": "^BVSP"},
}

In [9]:
start_date = "2025-01-01"
end_date = "2025-05-01"


data = {}
for country, ticker in world_indices.items():
    df = yf.download(ticker["ticker"], start=start_date, end=end_date, progress=False)
    data[country] = pl.from_pandas(
        df.reset_index(), schema_overrides={"('Date', '')": pl.Date}
    )
    data[country].columns = ["date", "close", "high", "low", "open", "volume"]


YF.download() has changed argument auto_adjust default to True


In [10]:
dfs_with_keys = [
    df.with_columns(pl.lit(key).alias("country")) for key, df in data.items()
]
combined_df = pl.concat(dfs_with_keys)

In [11]:
df_ytd = combined_df.group_by("country", maintain_order=True).agg(
    ytd=100
    * (pl.col("close").last() - pl.col("close").first())
    / pl.col("close").first()
)

In [12]:
df_ytd.filter(
    pl.col("ytd") > pl.col("ytd").filter(pl.col("country") == "United States").first()
).height

9

Question 3

In [13]:
sp500 = yf.download("^GSPC", start="1950-01-01", end="2025-05-01", progress=False)
df = pl.DataFrame(sp500.reset_index(), schema_overrides={"('Date', '')": pl.Date})
df.columns = ["date", "close", "high", "low", "open", "volume"]

In [14]:
df = (
    df.with_columns(pl.col("close").cum_max().alias("all_time_high"))
    .with_columns(
        pl.when(pl.col("close") == pl.col("all_time_high"))
        .then(pl.col("date"))
        .alias("high_date")
    ).with_columns(pl.col("high_date").fill_null(strategy="forward"))
    .with_columns(
        (pl.col("all_time_high") - pl.col("close")).alias("drawdown"),
    )
    .with_columns(
        ((pl.col("drawdown") / pl.col("all_time_high")) * 100).alias("drawdown_pct")
    )
)


In [15]:
corrections_duration = (
    df.filter(pl.col("drawdown_pct") >= 5)
    .group_by("high_date", maintain_order=True)
    .agg(
        pl.col("date")
        .filter(pl.col("drawdown_pct") == pl.col("drawdown_pct").max())
        .first()
        .alias("end_date_correction")
    )
    .with_columns(
        (pl.col("end_date_correction") - pl.col("high_date"))
        .dt.total_days()
        .alias("correction_duration_days")
    )
)

In [16]:
percentiles = corrections_duration.select(
    [
        pl.col("correction_duration_days").quantile(0.25).alias("25th_percentile"),
        pl.col("correction_duration_days").median().alias("50th_percentile"),
        pl.col("correction_duration_days").quantile(0.75).alias("75th_percentile"),
    ]
)
percentiles

25th_percentile,50th_percentile,75th_percentile
f64,f64,f64
22.0,40.5,87.0


Question 4

In [17]:
df_earning = pl.read_csv(
    Path("./../../../cohorts/2025/ha1_Amazon.csv"),
    separator=";",
    null_values="-",
    try_parse_dates=True,
).filter(~pl.all_horizontal(pl.all().is_null()))

In [18]:
df_earning = df_earning.with_columns(
    pl.when(pl.col("EPS Estimate").cast(pl.Utf8).str.contains(r"\?\?\?"))
    .then(None)
    .otherwise(pl.col("EPS Estimate"))
    .alias("EPS Estimate"),
    pl.when(pl.col("Reported EPS").cast(pl.Utf8).str.contains(r"\?\?\?"))
    .then(None)
    .otherwise(pl.col("Reported EPS"))
    .alias("Reported EPS"),
).with_columns(
    pl.col("EPS Estimate").cast(pl.Float64), pl.col("Reported EPS").cast(pl.Float64)
)

In [19]:
tzinfos = {
    "EST": pytz.timezone("US/Eastern"),
    "EDT": pytz.timezone("US/Eastern"),
}

In [20]:
df_earning = df_earning.with_columns(
    pl.col("Earnings Date")
    .map_elements(
        lambda x: parser.parse(x, tzinfos=tzinfos).strftime("%B %d, %Y at %I:%M %p %Z"),
        return_dtype=pl.String
    )
    .str.to_datetime(format="%B %d, %Y at %I:%M %p %Z")
    .alias("Earnings Date fixed")
).with_columns(
    pl.when(pl.col("Surprise (%)") > 0).then(1).otherwise(0).alias("surprise_earning"),
    pl.col("Earnings Date fixed").dt.date().alias("earning_date_only"),
)

In [21]:
df_amzn = yf.download("AMZN", progress=False)

In [22]:
df_amzn = pl.DataFrame(
    df_amzn.reset_index(), schema_overrides={"('Date', '')": pl.Date}
)
df_amzn.columns = ["date", "close", "high", "low", "open", "volume"]

In [23]:
df_amzn = df_amzn.with_columns(
    (pl.col("close").shift(-2) / pl.col("close") - 1).alias("pct_change_2d")
)


In [24]:
df_merged = df_earning.join(
    df_amzn, left_on="earning_date_only", right_on="date", how="left"
)

In [25]:
df_merged.filter(pl.col("surprise_earning") == 1).select(
    pl.col("pct_change_2d").median()
)

pct_change_2d
f64
0.002672


In [26]:
df_merged.select(pl.col("pct_change_2d").median())

pct_change_2d
f64
-0.015208
